In [14]:
import numpy as np
import pandas as pd
from scipy.stats import chi2_contingency
from apyori import apriori
Dataset = [
    [1000,100,100,100000], [1000, 100,100, 10000], [1000,100,100, 1000], [1000,100,100,0], [1000,1000,1000,1000], [100,100,1000,100000], [1000,100,10000,100000],[1,1,100,10000]
]
stat, p, dof, expected = chi2_contingency(Dataset)
alpha = 0.5
print("P value is "+ str(p))
if p <= alpha:
    print("Dependent (rejected)")
else:
    print("Independent Accepted")
obj = np.array(Dataset).T
print("\nChisquare of every row is :\n",chisquare(obj))


P value is 0.0
Dependent (rejected)

Chisquare of every row is :
 Power_divergenceResult(statistic=array([294097.23320158,  24878.57142857,   1472.72727273,   2200.        ,
            0.        , 294097.23320158, 252572.72727273,  29498.07998416]), pvalue=array([0., 0., 0., 0., 1., 0., 0., 0.]))


In [1]:
pip install tabulate

Note: you may need to restart the kernel to use updated packages.


In [23]:
#confidence
import scipy.stats as st
import numpy as np
st.t.interval(alpha = 0.99, df = len(Dataset)-1, loc = np.mean(Dataset), scale = st.sem(Dataset))

(array([ 10149.01390754,  10291.6434737 ,   6445.7956393 , -50710.29661419]),
 array([11238.61109246, 11095.9815263 , 14941.8293607 , 72097.92161419]))

In [1]:
#cosine
import scipy.stats as st
import numpy as np
def cosine(int milk, int coffee):

SyntaxError: invalid syntax (1011373803.py, line 4)